# Projeto 1 - Ciência dos Dados

Nome: Venâncio Freitas de Araújo Filho

Nome: WELLINGTON RODRIGUES DA SILVA

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\venan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\venan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\venan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\venan\OneDrive - Insper - Institudo de Ensino e Pesquisa\Materias 2° semestre Insper\Ciência de dados\Projetos\Projeto 1


Dados de Treino: 

Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_excel('dados_treino.xlsx')
train

,Mensagem,Target
0,Muito erros grosseiros de tradução. O material...,Editora
1,"Primeiramente meu pedido nem veio,Mas me reemb...",Amazon
2,"Livro com um monte de clichês, mas com pouco c...",Editora
3,Sim.adorei o livro Vou continuar a comprar o r...,Editora
4,Não recebi o livro . Não sei o motivo. Foi fei...,Amazon
...,...,...
293,O livro foi classificado como novo no site e c...,Amazon
294,"Não gostei porquê a capa veio com um rasgo, o ...",Amazon
295,EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...,Amazon
296,Não consigo entender o alto valor desses e-boo...,Amazon


In [4]:
train.Target.value_counts(True).round(3)

Autor      0.377
Editora    0.311
Amazon     0.311
Name: Target, dtype: float64

Dados de Teste


In [5]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,Amontoado de proposições absurdas destinadas a...,Autor
1,Quando comprei esse livro fui com uma grande e...,Autor
2,Li um terço do livro. Arrastei-me para procura...,Autor
3,"Sensacionalista, raso e cheio de distorções ma...",Autor
4,A mensagem do livro da pra entender em 1 págin...,Autor


In [6]:
test.Target.value_counts()

Autor      102
Editora     76
Amazon      33
Name: Target, dtype: int64

In [7]:
test.Target

0      Autor
1      Autor
2      Autor
3      Autor
4      Autor
       ...  
206    Autor
207    Autor
208    Autor
209    Autor
210    Autor
Name: Target, Length: 211, dtype: object

___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

## 1.Vamos Iniciar Extraindo os Dados Referente à cada rótulo

## Função CleanUp 1 :

In [8]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

## Função Clean UP 2: Stemming

In [9]:
def steamming(texto): #Recebe lista 
    stemmed_palavras = []
    stemmer = PorterStemmer()
    for palavra in texto:
        
        stemmed_palavras.append(stemmer.stem(palavra)) 
    
    return stemmed_palavras


## Função Clean UP 3: Stop Words

In [10]:
def stopwords2(texto): #Recebe lista 
    palavras_sem_stopwords = []
    palavras = word_tokenize(' '.join(texto), language='portuguese')

    stopwords_pt = set(stopwords.words('portuguese'))
    
    for palavra in palavras:
        if palavra not in stopwords_pt:
            palavras_sem_stopwords.append(palavra)
    
    return palavras_sem_stopwords

## Função Clean UP 4:  Remoção de Emojis

In [11]:
def remove_emojis(texto): #Recebe lista 
    texto = ' '.join(texto)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas diversos
                           u"\U0001F700-\U0001F77F"  # Símbolos alquímicos
                           u"\U0001F780-\U0001F7FF"  # Símbolos de palavras
                           u"\U0001F800-\U0001F8FF"  # Símbolos de árabes estendidos-A
                           u"\U0001F900-\U0001F9FF"  # Símbolos suplementares de árabes estendidos-B
                           u"\U0001FA00-\U0001FA6F"  # Símbolos de xadrez
                           u"\U0001FA70-\U0001FAFF"  # Símbolos suplementares de xadrez
                           u"\U0001F004-\U0001F0CF"  # Símbolos de domino
                           u"\U0001F170-\U0001F251"  # Símbolos de tai-xi
                           u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas diversos
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F680-\U0001F6FF"  # Transporte e mapas
                           u"\U0001F700-\U0001F77F"  # Símbolos alquímicos
                           u"\U0001F780-\U0001F7FF"  # Símbolos de palavras
                           u"\U0001F800-\U0001F8FF"  # Símbolos de árabes estendidos-A
                           u"\U0001FA00-\U0001FA6F"  # Símbolos de xadrez
                           u"\U0001FA70-\U0001FAFF"  # Símbolos suplementares de xadrez
                           "]+", flags=re.UNICODE)
    emoji_pattern = emoji_pattern.sub(r'', texto)
    emoji_pattern = emoji_pattern.split(' ')
    return emoji_pattern


## Função 5: Tokenização Por bigramas

In [12]:
def tokenize(texto):
    ' '.join(texto)
    lista_bigramas = list(bigrams(texto))
    return lista_bigramas

## Iniciando a extração dos textos referentes aos Rótulos

1. Dados do Autor 

In [13]:
dados_autor  = train.loc[train.Target == 'Autor' , :]


In [14]:
#Montando um Texto
texto_comentários_autor = ''
for i in dados_autor.Mensagem: 
    texto_comentários_autor += i
    texto_comentários_autor += ' '
texto_comentários_autor = cleanup(texto_comentários_autor)
texto_comentarios_autor = texto_comentários_autor.lower()
texto_comentarios_autor = texto_comentarios_autor.split(' ')
texto_comentarios_autor = remove_emojis(texto_comentarios_autor)
#texto_comentarios_autor = stopwords2(texto_comentarios_autor)
texto_comentarios_autor = steamming(texto_comentarios_autor)
#texto_comentarios_autor = tokenize(texto_comentarios_autor)
tabela_autor = pd.Series(texto_comentarios_autor )
tabela_autor.head()
print(tabela_autor.shape)

(6683,)


Tabelas Relativas ao autor 

In [15]:
#Frequências Absolutas 
tabela_autor_abs = tabela_autor.value_counts()
tabela_autor_abs

de             256
o              237
que            223
e              202
a              196
              ... 
invençõ          1
passaria         1
praticament      1
apontar          1
sentimento       1
Length: 1848, dtype: int64

In [16]:
#Frequências relativas 
tabela_autor_relativa = tabela_autor.value_counts(True)
tabela_autor_relativa 

de             0.038306
o              0.035463
que            0.033368
e              0.030226
a              0.029328
                 ...   
invençõ        0.000150
passaria       0.000150
praticament    0.000150
apontar        0.000150
sentimento     0.000150
Length: 1848, dtype: float64

In [17]:
tabela_autor_abs.sum()

6683

## 2. Dados da Editora 

In [18]:
dados_editora = train.loc[train.Target == 'Editora' , :]
dados_editora.shape

(90, 2)

In [19]:
comentarios_editora = dados_editora.Mensagem
texto_editora = ''
for e in comentarios_editora:
    texto_editora += e 
    texto_editora += ' '
texto_editora = cleanup(texto_editora)
texto_editora = texto_editora.lower()
texto_editora = texto_editora.split(' ')
texto_editora = remove_emojis(texto_editora)
#texto_editora = stopwords2(texto_editora)
texto_editora = steamming(texto_editora)
#texto_editora = tokenize(texto_editora)
tabela_editora = pd.Series(texto_editora)
print(tabela_editora.head())
print(tabela_editora.shape)

0        muito
1         erro
2    grosseiro
3           de
4     tradução
dtype: object
(4727,)


Frequências relativas às tabelas

In [20]:
#Frequência absoluta
tabela_editora_abs = tabela_editora.value_counts()

In [21]:
#Frequência Relativa 
tabela_editora_relativa = tabela_editora.value_counts(True)
tabela_editora_relativa

de           0.041041
a            0.033425
o            0.029406
que          0.028348
e            0.027925
               ...   
avaliação    0.000212
reduza       0.000212
dezena       0.000212
resisti      0.000212
frustada     0.000212
Length: 1520, dtype: float64

In [22]:
#Quantidade de palavras 
tabela_editora_abs.sum()

4727

3. Dados da Amazon


In [23]:
dados_amazon = train.loc[train.Target == 'Amazon' , :]
print(dados_amazon)
print(dados_amazon.shape)

                                              Mensagem  Target
1    Primeiramente meu pedido nem veio,Mas me reemb...  Amazon
4    Não recebi o livro . Não sei o motivo. Foi fei...  Amazon
7    Pelo amor de Deus!!!! que preço é este......só...  Amazon
12   FiZ a compra errada, era pra comprar o livro e...  Amazon
23   Não recebi o livro... Acredito que tenham envi...  Amazon
..                                                 ...     ...
293  O livro foi classificado como novo no site e c...  Amazon
294  Não gostei porquê a capa veio com um rasgo, o ...  Amazon
295  EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...  Amazon
296  Não consigo entender o alto valor desses e-boo...  Amazon
297  a caixa veio rasgada e amassada, gostaria de u...  Amazon

[90 rows x 2 columns]
(90, 2)


In [24]:
comentarios_amazon = dados_amazon.Mensagem
texto_amazon = ''
for texto in comentarios_amazon:
    texto_amazon += texto 
    texto_amazon += ' '
texto_amazon = cleanup(texto_amazon)
texto_amazon = texto_amazon.lower()
texto_amazon = texto_amazon.split(' ')
texto_amazon = remove_emojis(texto_amazon)
#texto_amazon = stopwords2(texto_amazon)
texto_amazon = steamming(texto_amazon)
#texto_amazon = tokenize(texto_amazon)
tabela_amazon = pd.Series(texto_amazon)

Frequências referentes à Amazon: 

In [25]:
#Frequência absoluta 
tabela_amazon_abs = tabela_amazon.value_counts()

In [26]:
#Frequência relativa
tabela_amazon_relativa = tabela_amazon.value_counts(True)
tabela_amazon_relativa

o              0.037732
a              0.033468
e              0.030484
que            0.027499
de             0.022170
                 ...   
cópia          0.000213
enviada        0.000213
lamppm         0.000213
responsável    0.000213
otimo          0.000213
Length: 1280, dtype: float64

In [27]:
tabela_amazon_abs.sum()

4691

___
### Construindo o Classificador a partir dos testes

Agora você deve testar o seu classificador com a base de Testes.

In [28]:
todas_palavras = texto_editora + texto_amazon + texto_comentarios_autor
lista_sem_repeticao = []
for palavra in todas_palavras: 
    if palavra not in lista_sem_repeticao: 
        lista_sem_repeticao.append(palavra)

# 1 Probabilidade de Cada Rótulo dentro do texto

In [29]:
P_Autor = sum(tabela_autor_abs) / len(todas_palavras)
P_Autor

0.41506738711881247

In [30]:
P_Amazon = sum(tabela_amazon_abs) / len(todas_palavras)
P_Amazon

0.291348363455686

In [31]:
P_Editora = sum(tabela_editora_abs) / len(todas_palavras)
P_Editora

0.29358424942550154

# 2. Probabilidade de aparecer em cada frase

In [32]:
P_frase_dado_autor = 1
P_frase_dado_editora = 1
P_frase_dado_amazon = 1
lista_teste = []
i = 0
lista_probs = []
alpha = 1

In [33]:
for frase in test.Mensagem:
    frase = cleanup(str(frase))
    frase = frase.lower()
    frase = frase.split(' ')
    frase = remove_emojis(frase)
    #frase = stopwords2(frase)
    frase = steamming(frase)
    #frase = tokenize(frase)
    for palavras in frase:
        
        if palavras in texto_comentarios_autor: 
            P_frase_dado_autor *= (tabela_autor_abs[palavras] + alpha)/ ( (len(lista_sem_repeticao)* alpha) + len(texto_comentarios_autor) )
        else: 
            P_frase_dado_autor *= alpha/(len(texto_comentarios_autor) +(len(lista_sem_repeticao)* alpha))
            
        
        if palavras in texto_amazon: 
            P_frase_dado_amazon *= (tabela_amazon_abs[palavras] + alpha)/(len(texto_amazon) + (len(lista_sem_repeticao)* alpha))
        else: 
            P_frase_dado_amazon *= alpha/(len(texto_amazon)+(len(lista_sem_repeticao)* alpha))
        
                
        if palavras in texto_editora: 
            P_frase_dado_editora *= (tabela_editora_abs[palavras] + alpha) / ((len(lista_sem_repeticao)* alpha) + len(texto_editora))
        else: 
            P_frase_dado_editora *= (alpha/ ((len(lista_sem_repeticao)* alpha) + len(texto_editora)))
            
        
            
    P_autor_dado_frase = P_frase_dado_autor * P_Autor
    lista_probs.append( P_autor_dado_frase)
    
    P_editora_dado_frase = P_frase_dado_editora * P_Editora
    lista_probs.append( P_editora_dado_frase)
    
    P_amazon_dado_frase =  P_frase_dado_amazon * P_Amazon
    lista_probs.append( P_amazon_dado_frase)
    
    if max(lista_probs) == lista_probs[0]:
        lista_teste.append('Autor')
    elif max(lista_probs) == lista_probs[1]:
        lista_teste.append('Editora')
    elif max(lista_probs) == lista_probs[2]:
        lista_teste.append('Amazon')
        
    P_frase_dado_editora = 1
    P_frase_dado_amazon = 1
    P_frase_dado_autor = 1
    
    P_autor_dado_frase = 1
    
    P_editora_dado_frase = 1
    
    P_amazon_dado_frase =  1
    lista_probs = []
    
    i += 1
    
lista_teste

['Editora',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Amazon',
 'Autor',
 'Editora',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Amazon',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Amazon',
 'Editora',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Amazon',
 'Amazon',
 'Autor',
 'Amazon',
 'Amazon',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Editora',
 'Editora',
 'Autor',
 'Amazon',
 'Amazon',
 'Autor',
 'Autor',
 'E

## 4. Agora, Vamos Verificar a Performance do Treino

# Verificando os verdadeiros positivos

In [34]:
test['Bot'] = lista_teste
tabela_verificação = (pd.crosstab(test.Target , test.Bot , normalize = True )*100).round(3)
tabela_verificação

Bot,Amazon,Autor,Editora
Target,,,
Amazon,14.692,0.474,0.474
Autor,1.422,44.550,2.370
Editora,4.739,21.327,9.953


# Verificando Acertos no Rótulo Autor 


# Verificando Acertos no Rótulo Editora

## Verificando Acertos no Rótulo Amazon

## Acurácia 

## 5. Agora, Vamos calcular os desvios

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**